# 2. 선형 회귀(Linear Regression)

## 2.1. 선형 회귀(Linear Regression)

---
### 2.1.1. 데이터에 대한 이해
- 훈련 데이터셋(training dataset) : 예측 훈련을 위해 사용하는 데이터셋
- 테스트 데이터셋(test dataset) : 모델이 얼마나 잘 작동하는지 판별하는 데이터셋

---
### 2.1.2. 가설(Hypothesis) 수립
- 보통은 임의의 추측으로 세우거나 경험적으로 알고 있는 식을 세운다  
  
   
- 선형 회귀
    - 학습 데이터와 가장 잘 맞는 하나의 직선을 찾는 일
    - y = Wx + b or H(x) = Wx + b
    - W : 가중치(weight), b : 편향(bias)

---
### 2.1.3. 비용 함수(Cost function)에 대한 이해
- 비용 함수 = 손실 함수(loss function) = 오차 함수(error function) = 목적 함수(objective function)

![cost_function](../image/cost_function.png)

위의 그래프에서 수식이 예측한 값과 실제값의 차이를 오차로 정의하면 음수가 나오는 경우가 생긴다. 그렇기에 구한 오차를 제곱하여 더해주고 데이터의 개수만큼 나누는 평균 제곱 오차(Mean Squared Error, MSE)를 사용한다.

![mse](../image/mse.png)

Cost(W, b)를 최소가 되게 만드는 W와 b를 구하면 훈련 데이터와 오차가 가장 적은 직선을 구할 수 있다.

---
### 2.1.4. 옵티마이저 - 경사 하강법(Gradient Descent)
- 옵티마이저(Optimizer) 알고리즘 = 최적화 알고리즘
- 옵티마이저 알고리즘을 통해 적당한 W, b를 구하는 과정을 학습(training)이라 부른다
- 경사 하강법은 가장 기본적인 옵티마이저 알고리즘이다

![경사하강1](../image/경사하강1.png)
![경사하강2](../image/경사하강2.png)

기울기가 무한대로 커지거나 작아지면 cost도 무한대로 커진다. 훈련의 목적은 cost가 가장 작은 값을 갖는 W를 찾는 것이므로 볼록한 부분의 W를 찾아야 한다. 그렇기에 접선의 기울기가 0에 가까운 방향으로 움직이며 W를 찾아야 한다.  
  
  
W값은 접선의 기울기가 음수일 땐 증가하여 0으로, 양수일 땐 감소하여 0으로 향하게 된다. 

![경사하강3](../image/경사하강3.png)

여기서 <math xmlns="http://www.w3.org/1998/Math/MathML">
  <mi>&#x3B1;</mi>
</math>는 학습률(learning rate)라고 부르며 W의 값이 얼마나 크게 이동할지 결정한다. 학습률이 크면 클수록 좋아보이지만 실상은 그렇진 않다.

![경사하강4](../image/경사하강4.png)

학습률이 지나치게 크면 위 그림처럼 W의 값이 발산하여 무한대로 가게된다. 반대로 학습률이 너무 작으면 학습 속도가 느려지므로 적당한 학습률을 찾는 것이 중요하다.

위 예시들은 b를 배제했으나 실제 사용할 땐 W와 b를 모두 활용하여 최적의 W, b를 찾아야 한다. 문제에 따라 적합한 가설, 비용 함수, 옵티마이저가 다 다르지만 선형 회귀에선 평균 제곱 오차가 가장 적합한 비용 함수이고 경사 하강법이 가장 적합한 옵티마이저이다.

---
### 2.1.5. 파이토치로 선형 회귀 구현하기

기본 세팅

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1) # 재실행해도 같은 결과가 나오도록 랜덤 시드를 준다.

변수 선언

In [11]:
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

print(x_train, x_train.shape)
print(y_train, y_train.shape)

tensor([[1.],
        [2.],
        [3.]]) torch.Size([3, 1])
tensor([[2.],
        [4.],
        [6.]]) torch.Size([3, 1])


가중치와 편향의 초기화

In [12]:
# 가중치 W를 0으로 초기화하고 학습을 통해 값이 변경되는 변수임을 명시
W = torch.zeros(1, requires_grad=True)
W

tensor([0.], requires_grad=True)

In [13]:
# 편향 b를 0으로 초기화하고 학습을 통해 값이 변경되는 변수임을 명시
b = torch.zeros(1, requires_grad=True)
b

tensor([0.], requires_grad=True)

- 현재 직선 방정식  
y = 0 * x + 0  
> x에 어떤 값이 들어가도 가설은 0을 예측한다. 그렇기에 아직 적절한 W와 b값이 아니다.

가설 세우기

In [14]:
# 직선 방정식으로 가설 선언
hypothesis = x_train * W + b
hypothesis

tensor([[0.],
        [0.],
        [0.]], grad_fn=<AddBackward0>)

비용 함수 선언하기

In [15]:
# MSE를 선언
cost = torch.mean((hypothesis - y_train) ** 2)
cost

tensor(18.6667, grad_fn=<MeanBackward0>)

경사 하강법 구현하기

In [16]:
# SGD :경사 하강법의 일종 / lr : 학습률(learning rate)
optimizer = optim.SGD([W, b], lr=0.01)

optimizer.zero_grad # 기울기를 0으로 초기화
cost.backward() # 비용 함수를 미분하여 기울기 계산
optimizer.step() # W와 b를 업데이트 / 리턴되는 변수들의 기울기에 학습룰을 곱하여 뺴줌으로 업데이트

전체 코드

In [18]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

# 모델 초기화
W = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.01)

nb_epochs = 2000 # 경사 하강법 반복 횟수
for epoch in range(nb_epochs + 1):
    # H(x) 계산
    hypothesis = x_train * W + b
    
    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print(f'Epoch {epoch:4d}/{nb_epochs} W : {W.item():.3f}, b : {b.item():.3f} Cost : {cost.item():.6f}')

Epoch    0/2000 W : 0.187, b : 0.080 Cost : 18.666666
Epoch  100/2000 W : 1.746, b : 0.578 Cost : 0.048171
Epoch  200/2000 W : 1.800, b : 0.454 Cost : 0.029767
Epoch  300/2000 W : 1.843, b : 0.357 Cost : 0.018394
Epoch  400/2000 W : 1.876, b : 0.281 Cost : 0.011366
Epoch  500/2000 W : 1.903, b : 0.221 Cost : 0.007024
Epoch  600/2000 W : 1.924, b : 0.174 Cost : 0.004340
Epoch  700/2000 W : 1.940, b : 0.136 Cost : 0.002682
Epoch  800/2000 W : 1.953, b : 0.107 Cost : 0.001657
Epoch  900/2000 W : 1.963, b : 0.084 Cost : 0.001024
Epoch 1000/2000 W : 1.971, b : 0.066 Cost : 0.000633
Epoch 1100/2000 W : 1.977, b : 0.052 Cost : 0.000391
Epoch 1200/2000 W : 1.982, b : 0.041 Cost : 0.000242
Epoch 1300/2000 W : 1.986, b : 0.032 Cost : 0.000149
Epoch 1400/2000 W : 1.989, b : 0.025 Cost : 0.000092
Epoch 1500/2000 W : 1.991, b : 0.020 Cost : 0.000057
Epoch 1600/2000 W : 1.993, b : 0.016 Cost : 0.000035
Epoch 1700/2000 W : 1.995, b : 0.012 Cost : 0.000022
Epoch 1800/2000 W : 1.996, b : 0.010 Cost : 0

2000번의 훈련 결과 최적의 기울기 W는 1.997, b는 0.006인 것을 확인할 수 있다. 실제 정답이 W = 2, b = 0이므로 거의 정답을 찾은 것이다.

---
### 2.1.6 optimizer.zero_grad()가 필요한 이유

In [19]:
import torch
w = torch.tensor(2.0, requires_grad=True)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    z = 2 * w
    z.backward()
    print('수식을 w로 미분한 값 : {}'.format(w.grad))

수식을 w로 미분한 값 : 2.0
수식을 w로 미분한 값 : 4.0
수식을 w로 미분한 값 : 6.0
수식을 w로 미분한 값 : 8.0
수식을 w로 미분한 값 : 10.0
수식을 w로 미분한 값 : 12.0
수식을 w로 미분한 값 : 14.0
수식을 w로 미분한 값 : 16.0
수식을 w로 미분한 값 : 18.0
수식을 w로 미분한 값 : 20.0
수식을 w로 미분한 값 : 22.0
수식을 w로 미분한 값 : 24.0
수식을 w로 미분한 값 : 26.0
수식을 w로 미분한 값 : 28.0
수식을 w로 미분한 값 : 30.0
수식을 w로 미분한 값 : 32.0
수식을 w로 미분한 값 : 34.0
수식을 w로 미분한 값 : 36.0
수식을 w로 미분한 값 : 38.0
수식을 w로 미분한 값 : 40.0
수식을 w로 미분한 값 : 42.0


파이토치는 미분을 통해 얻은 기울기를 이전에 계산된 기울기 값에 누적시키는 특징이 있다. 위의 결과를 보면 기울기 2가 계속하여 누적되는 것을 확인할 수 있다. 그렇기에 미분값을 계속 0으로 초기화해주어야 한다.

---
### 2.1.7 torch.manual_seed()를 하는  이유

torch.manual_seed()를 사용한 프로그램의 결과는 어느 곳에서 실행시켜도 동일한 결과를 얻을 수 있다. 그 이유는 난수 발생 순서와 값을 동일하게 보장해준다는 특징이 있기 때문이다. 

In [21]:
import torch

torch.manual_seed(3)
print('랜덤 시드가 3일 때')
for i in range(1, 3):
    print(torch.rand(1))
    
print()
torch.manual_seed(5)
print('랜덤 시드가 5일 때')
for i in range(1, 3):
    print(torch.rand(1))
    
print()
torch.manual_seed(3)
print('랜덤 시드가 다시 3일 때')
for i in range(1, 3):
    print(torch.rand(1))

랜덤 시드가 3일 때
tensor([0.0043])
tensor([0.1056])

랜덤 시드가 5일 때
tensor([0.8303])
tensor([0.1261])

랜덤 시드가 다시 3일 때
tensor([0.0043])
tensor([0.1056])
